In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.6.0+cu124


In [2]:
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 1.4 MB/s eta 0:00:00


In [3]:
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import torch
# import util
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from numpy import asarray
import tifffile as tiff
import torch.nn as nn
import torch.nn.functional as nnFn
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from torch.nn import CrossEntropyLoss
import random
from sklearn.cluster import KMeans

In [4]:
fa_feature_path = "/content/drive/MyDrive/TejaswiAbburi_va797/Dataset/ISBI_ADNI_CN_dataset/Processed_histogram_features_CN_All/Histogram_CN_FA_20bin_updated.npy"
Histogram_feature_CN_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Load AD features
fa_feature_path = "/content/drive/MyDrive/TejaswiAbburi_va797/Dataset/ISBI_ADNI_AD_dataset/Processed_histogram_features_AD_All/Histogram_AD_FA_20bin_updated.npy"
Histogram_feature_AD_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Combine features and labels
X = np.vstack([Histogram_feature_CN_FA_array, Histogram_feature_AD_FA_array])
y = np.hstack([
    np.zeros(Histogram_feature_CN_FA_array.shape[0], dtype=np.int64),
    np.ones(Histogram_feature_AD_FA_array.shape[0], dtype=np.int64)
])
np.random.seed(42)
perm = np.random.permutation(X.shape[0])
X = X[perm]
y = y[perm]
num_nodes, num_feats = X.shape
print(f"Features: {X.shape}, Labels: {y.shape}")

Features: (223, 180), Labels: (223,)


In [5]:
# F = np.concatenate((Histogram_feature_CN_FA_array, Histogram_feature_MCI_FA_array), axis=0)
# F = F.astype(np.float32)
# print("Combined Shape:", F.shape)
F = X
F = F.astype(np.float32)
print("Final Shape:", F.shape)
print("Data Type:", F.dtype)

Final Shape: (223, 180)
Data Type: float32


In [6]:
kmeans = KMeans(n_clusters=2, random_state=11, max_iter=5000)
kmeans.fit(F)

# Cluster probabilities
klabels_trans = kmeans.transform(F)
klabels_trans = klabels_trans / klabels_trans.sum(axis=1, keepdims=True)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kS = torch.tensor(klabels_trans, dtype=torch.float32).to(device)

In [8]:
ground_truth_tensor = torch.tensor(y, dtype=torch.long).to(device)

In [9]:
criterion = CrossEntropyLoss()
loss = criterion(kS, ground_truth_tensor)
y_pred = torch.argmax(kS, dim=1).cpu().numpy()

In [10]:
acc_score = accuracy_score(y, y_pred)
acc_score_inverted = accuracy_score(y, 1 - y_pred)

if acc_score_inverted > acc_score:
    acc_score = acc_score_inverted
    y_pred = 1 - y_pred
    klabels_trans[:, [0, 1]] = klabels_trans[:, [1, 0]]

y_pred_proba = klabels_trans[:, 1]

prec_score = precision_score(y, y_pred)
rec_score = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)
log_loss_value = log_loss(y, y_pred_proba)

# Print metrics
print("Accuracy Score:", acc_score)
print("Precision Score:", prec_score)
print("Recall Score:", rec_score)
print("F1 Score:", f1)
print("Log Loss:", log_loss_value)

Accuracy Score: 0.7713004484304933
Precision Score: 0.76
Recall Score: 0.6333333333333333
F1 Score: 0.6909090909090909
Log Loss: 0.5972930360409032


In [11]:
print(y_pred_proba)

[0.39955494 0.3900371  0.40605935 0.5571769  0.4419918  0.5925987
 0.44482198 0.4673245  0.48940197 0.5986793  0.3379363  0.41085538
 0.5667939  0.37261823 0.4098041  0.51671594 0.35396242 0.41272566
 0.4832051  0.36676618 0.47465894 0.4116862  0.39721495 0.6003659
 0.5143832  0.6259099  0.51990634 0.42947254 0.37456074 0.37188953
 0.39308304 0.413936   0.35067508 0.634193   0.6123798  0.36423844
 0.46656772 0.40643543 0.42304927 0.57930714 0.63052887 0.40642554
 0.5967476  0.47446635 0.5580478  0.39498228 0.39293194 0.3749694
 0.5864426  0.60091585 0.4587199  0.5424537  0.58688337 0.46541774
 0.34578288 0.4043524  0.4766571  0.4273038  0.66668814 0.36352104
 0.5279089  0.36777103 0.38821045 0.41968563 0.39618766 0.57056606
 0.62325096 0.3757967  0.49485922 0.48154294 0.37381727 0.41078112
 0.3671435  0.47247684 0.48430622 0.36378235 0.39693433 0.36596802
 0.3448817  0.5544861  0.4560612  0.62017816 0.5769359  0.33459494
 0.5609579  0.6309949  0.44483203 0.38720173 0.53622675 0.4156798

In [13]:
import numpy as np
import torch
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from torch.nn import CrossEntropyLoss

fa_feature_path = "/content/drive/MyDrive/TejaswiAbburi_va797/Dataset/ISBI_ADNI_CN_dataset/Processed_histogram_features_CN_All/Histogram_CN_FA_20bin_updated.npy"
Histogram_feature_CN_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Load AD features
fa_feature_path = "/content/drive/MyDrive/TejaswiAbburi_va797/Dataset/ISBI_ADNI_AD_dataset/Processed_histogram_features_AD_All/Histogram_AD_FA_20bin_updated.npy"
Histogram_feature_AD_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Combine features and labels
X = np.vstack([Histogram_feature_CN_FA_array, Histogram_feature_AD_FA_array])
y = np.hstack([
    np.zeros(Histogram_feature_CN_FA_array.shape[0], dtype=np.int64),
    np.ones(Histogram_feature_AD_FA_array.shape[0], dtype=np.int64)
])
np.random.seed(42)
perm = np.random.permutation(X.shape[0])
X = X[perm]
y = y[perm]
num_nodes, num_feats = X.shape
print(f"Features: {X.shape}, Labels: {y.shape}")
F = X.astype(np.float32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = CrossEntropyLoss()

num_runs = 10

acc_scores = []
prec_scores = []
rec_scores = []
f1_scores = []
log_losses = []
ce_losses = [] # Initialize ce_losses list

for run in range(num_runs):
    print(f"\n--- Run {run+1}/{num_runs} ---")
    np.random.seed(run)
    torch.manual_seed(run)

    kmeans = KMeans(n_clusters=2, random_state=run, max_iter=5000)
    kmeans.fit(F)

    klabels_trans = kmeans.transform(F)
    klabels_trans = klabels_trans / klabels_trans.sum(axis=1, keepdims=True)

    kS = torch.tensor(klabels_trans, dtype=torch.float32).to(device)
    ground_truth_tensor = torch.tensor(y, dtype=torch.long).to(device)


    ce_loss_value = criterion(kS, ground_truth_tensor).item()


    y_pred = torch.argmax(kS, dim=1).cpu().numpy()
    acc_score = accuracy_score(y, y_pred)

    acc_score_inverted = accuracy_score(y, 1 - y_pred)
    if acc_score_inverted > acc_score:
        acc_score = acc_score_inverted
        y_pred = 1 - y_pred
        klabels_trans[:, [0, 1]] = klabels_trans[:, [1, 0]]

    y_pred_proba = klabels_trans[:, 1]

    prec_score = precision_score(y, y_pred)
    rec_score = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    log_loss_value = log_loss(y, y_pred_proba)

    acc_scores.append(acc_score)
    prec_scores.append(prec_score)
    rec_scores.append(rec_score)
    f1_scores.append(f1)
    log_losses.append(log_loss_value)
    ce_losses.append(ce_loss_value)

    print(f"Run {run+1} | Acc: {acc_score:.4f} | Prec: {prec_score:.4f} | Rec: {rec_score:.4f} | "
          f"F1: {f1:.4f} | LogLoss: {log_loss_value:.4f} | CE Loss: {ce_loss_value:.4f}")

print("\n================ FINAL SUMMARY ================\n")
print(f"{'Metric':>15} | {'Mean':>10} ± {'Std':<10}")
print("-" * 50)
print(f"{'Accuracy':>15} | {np.mean(acc_scores):.4f} ± {np.std(acc_scores):.4f}")
print(f"{'Precision':>15} | {np.mean(prec_scores):.4f} ± {np.std(prec_scores):.4f}")
print(f"{'Recall':>15} | {np.mean(rec_scores):.4f} ± {np.std(rec_scores):.4f}")
print(f"{'F1 Score':>15} | {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"{'Log Loss':>15} | {np.mean(log_losses):.4f} ± {np.std(log_losses):.4f}")
print(f"{'CE Loss':>15} | {np.mean(ce_losses):.4f} ± {np.std(ce_losses):.4f}")

Features: (223, 180), Labels: (223,)

--- Run 1/10 ---
Run 1 | Acc: 0.7713 | Prec: 0.7600 | Rec: 0.6333 | F1: 0.6909 | LogLoss: 0.5973 | CE Loss: 0.7547

--- Run 2/10 ---
Run 2 | Acc: 0.7713 | Prec: 0.7468 | Rec: 0.6556 | F1: 0.6982 | LogLoss: 0.5995 | CE Loss: 0.6425

--- Run 3/10 ---
Run 3 | Acc: 0.7713 | Prec: 0.7600 | Rec: 0.6333 | F1: 0.6909 | LogLoss: 0.5973 | CE Loss: 0.7547

--- Run 4/10 ---
Run 4 | Acc: 0.7713 | Prec: 0.7468 | Rec: 0.6556 | F1: 0.6982 | LogLoss: 0.5995 | CE Loss: 0.7532

--- Run 5/10 ---
Run 5 | Acc: 0.7713 | Prec: 0.7600 | Rec: 0.6333 | F1: 0.6909 | LogLoss: 0.5973 | CE Loss: 0.7547

--- Run 6/10 ---
Run 6 | Acc: 0.7713 | Prec: 0.7468 | Rec: 0.6556 | F1: 0.6982 | LogLoss: 0.5995 | CE Loss: 0.6425

--- Run 7/10 ---
Run 7 | Acc: 0.7713 | Prec: 0.7600 | Rec: 0.6333 | F1: 0.6909 | LogLoss: 0.5973 | CE Loss: 0.7547

--- Run 8/10 ---
Run 8 | Acc: 0.7713 | Prec: 0.7468 | Rec: 0.6556 | F1: 0.6982 | LogLoss: 0.5995 | CE Loss: 0.7532

--- Run 9/10 ---
Run 9 | Acc: 0.77